In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tqdm import tqdm


# Keras 
from keras.models import Sequential
from keras.layers import LSTM, GRU, SimpleRNN
from keras.layers import Dense, Activation, Dropout
from keras.layers import Embedding
from keras.layers import BatchNormalization
from keras.utils import np_utils, pad_sequences
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from keras.layers import GlobalMaxPooling1D, Conv1D, Flatten, MaxPooling1D, Bidirectional, SpatialDropout1D
from keras.callbacks import EarlyStopping

# Sklearn
from keras.preprocessing import sequence, text

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from plotly import graph_objs as go
import plotly.express as px
import plotly.figure_factory as ff

/opt/conda/lib/python3.7/site-packages/geopandas/_compat.py:115: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [2]:
## TPU Configuration

# Detect hardware, return appropriate distribution strategy
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

Running on TPU  grpc://10.0.0.2:8470
REPLICAS:  8


In [3]:
train = pd.read_csv('/kaggle/input/jigsaw-multilingual-toxic-comment-classification/jigsaw-toxic-comment-train.csv')
validation = pd.read_csv('/kaggle/input/jigsaw-multilingual-toxic-comment-classification/validation.csv')
test = pd.read_csv('/kaggle/input/jigsaw-multilingual-toxic-comment-classification/test.csv')
display(train.head())
display(test.head())

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


,id,content,lang
0,0,Doctor Who adlı viki başlığına 12. doctor olar...,tr
1,1,"Вполне возможно, но я пока не вижу необходимо...",ru
2,2,"Quindi tu sei uno di quelli conservativi , ...",it
3,3,Malesef gerçekleştirilmedi ancak şöyle bir şey...,tr
4,4,:Resim:Seldabagcan.jpg resminde kaynak sorunu ...,tr


In [4]:
train.drop(['severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'], axis =1 , inplace = True)
train.head()

,id,comment_text,toxic
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0


In [5]:
train = train.loc[:12000, :]
train.shape

(12001, 3)

In [6]:
train.comment_text.apply(lambda x: len(str(x).split())).max()

1403

In [7]:
def roc_auc(predictions, target):
    
    fpr, tpr, thresholds= metrics.roc_curve(target, predictions)
    roc_auc = metrics.auc(fpr, tpr)
    return roc_auc

In [8]:
xtrain, xvalid, ytrain, yvalid = model_selection.train_test_split(train.comment_text.values, train.toxic.values, stratify=train.toxic.values, shuffle = True, test_size=0.2)

In [9]:
tokenizer = text.Tokenizer(num_words = None)
max_len = 1500

tokenizer.fit_on_texts(list(xtrain) + list(xvalid))
xtrain_seq = tokenizer.texts_to_sequences(xtrain)
xvalid_seq = tokenizer.texts_to_sequences(xvalid)

# Padding
xtrain_pad = pad_sequences(xtrain_seq, maxlen=max_len)
xvalid_pad = pad_sequences(xvalid_seq, maxlen=max_len)

word_index = tokenizer.word_index

In [10]:
# word_index

In [11]:
%%time
## SimpleRNN

with strategy.scope():
    # A simpleRNN without any pretrained embeddings and one dense layer
    model = Sequential()
    model.add(Embedding(len(word_index)+1, 300, input_length=max_len))
    model.add(SimpleRNN(100))
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 1500, 300)         13049100  
                                                                 
 simple_rnn (SimpleRNN)      (None, 100)               40100     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 13,089,301
Trainable params: 13,089,301
Non-trainable params: 0
_________________________________________________________________
CPU times: user 318 ms, sys: 52.5 ms, total: 370 ms
Wall time: 2.26 s


In [12]:
model.fit(xtrain_pad, ytrain, epochs=5, batch_size=64*strategy.num_replicas_in_sync)

Epoch 1/5
19/19 [==============================] - 12s 274ms/step - loss: 0.3810 - accuracy: 0.8631
Epoch 2/5
19/19 [==============================] - 1s 70ms/step - loss: 0.2959 - accuracy: 0.9053
Epoch 3/5
19/19 [==============================] - 1s 70ms/step - loss: 0.2694 - accuracy: 0.8936
Epoch 4/5
19/19 [==============================] - 1s 71ms/step - loss: 0.2111 - accuracy: 0.9082
Epoch 5/5
19/19 [==============================] - 1s 73ms/step - loss: 0.1605 - accuracy: 0.9383


In [13]:
scores = model.predict(xvalid_pad)
print("AUC: %.2f%%" %roc_auc(scores, yvalid))

76/76 [==============================] - 3s 24ms/step
AUC: 0.76%


In [14]:
scores_model = []
scores_model.append({'Model': 'SimpleRNN', 'AUC': roc_auc(scores, yvalid)})

In [15]:
# Importing GLOVE Embeddings
# load the GloVe vectors in a dictionary:

embeddings_index = {}
f = open('/kaggle/input/glove840b300dtxt/glove.840B.300d.txt','r',encoding='utf-8')
for line in tqdm(f):
    values = line.split(' ')
    word = values[0]
    coefs = np.asarray([float(val) for val in values[1:]])
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

2196018it [04:36, 7950.21it/s]

Found 2196017 word vectors.


In [16]:
# create an embedding matrix for the words we have in the dataset
embedding_matrix = np.zeros((len(word_index) + 1, 300))
for word, i in tqdm(word_index.items()):
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

100%|██████████| 43496/43496 [00:00<00:00, 251419.05it/s]


## LSTM

In [17]:
%%time
with strategy.scope():
    
    # A simple LSTM with glove embeddings and one dense layer
    model = Sequential()
    model.add(Embedding(len(word_index) + 1,
                     300,
                     weights=[embedding_matrix],
                     input_length=max_len,
                     trainable=False))

    model.add(LSTM(100, dropout=0.3, recurrent_dropout=0.3))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])
    
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 1500, 300)         13049100  
                                                                 
 lstm (LSTM)                 (None, 100)               160400    
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                                 
Total params: 13,209,601
Trainable params: 160,501
Non-trainable params: 13,049,100
_________________________________________________________________
CPU times: user 809 ms, sys: 1.1 s, total: 1.91 s
Wall time: 2.71 s


In [18]:
model.fit(xtrain_pad, ytrain, epochs=5, batch_size=64*strategy.num_replicas_in_sync)

Epoch 1/5
19/19 [==============================] - 13s 322ms/step - loss: 0.3524 - accuracy: 0.8733
Epoch 2/5
19/19 [==============================] - 3s 169ms/step - loss: 0.2303 - accuracy: 0.9200
Epoch 3/5
19/19 [==============================] - 3s 172ms/step - loss: 0.1696 - accuracy: 0.9390
Epoch 4/5
19/19 [==============================] - 3s 169ms/step - loss: 0.1430 - accuracy: 0.9478
Epoch 5/5
19/19 [==============================] - 3s 169ms/step - loss: 0.1287 - accuracy: 0.9521


In [19]:
scores = model.predict(xvalid_pad)
print("Auc: %.2f%%" % (roc_auc(scores,yvalid)))

76/76 [==============================] - 5s 54ms/step
Auc: 0.94%


In [20]:
scores_model.append({'Model': 'LSTM','AUC_Score': roc_auc(scores,yvalid)})

## GRU

In [21]:
%%time
with strategy.scope():
    # GRU with glove embeddings and two dense layers
     model = Sequential()
     model.add(Embedding(len(word_index) + 1,
                     300,
                     weights=[embedding_matrix],
                     input_length=max_len,
                     trainable=False))
     model.add(SpatialDropout1D(0.3))
     model.add(GRU(300))
     model.add(Dense(1, activation='sigmoid'))

     model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])   
    
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 1500, 300)         13049100  
                                                                 
 spatial_dropout1d (SpatialD  (None, 1500, 300)        0         
 ropout1D)                                                       
                                                                 
 gru (GRU)                   (None, 300)               541800    
                                                                 
 dense_2 (Dense)             (None, 1)                 301       
                                                                 
Total params: 13,591,201
Trainable params: 542,101
Non-trainable params: 13,049,100
_________________________________________________________________
CPU times: user 912 ms, sys: 885 ms, total: 1.8 s
Wall time: 3.16 s


In [22]:
model.fit(xtrain_pad, ytrain, epochs=5, batch_size=64*strategy.num_replicas_in_sync)

Epoch 1/5
19/19 [==============================] - 11s 277ms/step - loss: 0.3169 - accuracy: 0.8931
Epoch 2/5
19/19 [==============================] - 3s 159ms/step - loss: 0.1964 - accuracy: 0.9366
Epoch 3/5
19/19 [==============================] - 3s 162ms/step - loss: 0.1520 - accuracy: 0.9508
Epoch 4/5
19/19 [==============================] - 3s 159ms/step - loss: 0.1251 - accuracy: 0.9561
Epoch 5/5
19/19 [==============================] - 3s 161ms/step - loss: 0.1140 - accuracy: 0.9583


In [23]:
scores = model.predict(xvalid_pad)
print("Auc: %.2f%%" % (roc_auc(scores,yvalid)))

76/76 [==============================] - 5s 49ms/step
Auc: 0.96%


In [24]:
scores_model.append({'Model': 'GRU','AUC_Score': roc_auc(scores,yvalid)})

In [25]:
scores_model

[{'Model': 'SimpleRNN', 'AUC': 0.756456966390948},
 {'Model': 'LSTM', 'AUC_Score': 0.938664391750321},
 {'Model': 'GRU', 'AUC_Score': 0.9562085357995371}]

## Bidirectional LSTM

In [26]:
%%time
with strategy.scope():
    # A simple bidirectional LSTM with glove embeddings and one dense layer
    model = Sequential()
    model.add(Embedding(len(word_index) + 1,
                     300,
                     weights=[embedding_matrix],
                     input_length=max_len,
                     trainable=False))
    model.add(Bidirectional(LSTM(300, dropout=0.3, recurrent_dropout=0.3)))

    model.add(Dense(1,activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])
    
    
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 1500, 300)         13049100  
                                                                 
 bidirectional (Bidirectiona  (None, 600)              1442400   
 l)                                                              
                                                                 
 dense_3 (Dense)             (None, 1)                 601       
                                                                 
Total params: 14,492,101
Trainable params: 1,443,001
Non-trainable params: 13,049,100
_________________________________________________________________
CPU times: user 1.01 s, sys: 811 ms, total: 1.82 s
Wall time: 3.11 s


In [27]:
model.fit(xtrain_pad, ytrain, epochs=5, batch_size=64*strategy.num_replicas_in_sync)

Epoch 1/5
19/19 [==============================] - 27s 841ms/step - loss: 0.3124 - accuracy: 0.8777
Epoch 2/5
19/19 [==============================] - 11s 596ms/step - loss: 0.1945 - accuracy: 0.9303
Epoch 3/5
19/19 [==============================] - 11s 598ms/step - loss: 0.1601 - accuracy: 0.9426
Epoch 4/5
19/19 [==============================] - 11s 597ms/step - loss: 0.1380 - accuracy: 0.9490
Epoch 5/5
19/19 [==============================] - 11s 595ms/step - loss: 0.1291 - accuracy: 0.9517


In [28]:
scores = model.predict(xvalid_pad)
print("Auc: %.2f%%" % (roc_auc(scores,yvalid)))
# Auc: 0.97%
scores_model.append({'Model': 'Bi-directional LSTM','AUC_Score': roc_auc(scores,yvalid)})

76/76 [==============================] - 10s 122ms/step
Auc: 0.94%


In [29]:
# Visualization of Results obtained from various Deep learning models
results = pd.DataFrame(scores_model).sort_values(by='AUC_Score',ascending=False)
results.style.background_gradient(cmap='Blues')

,Model,AUC,AUC_Score
2,GRU,nan,0.956209
1,LSTM,nan,0.938664
3,Bi-directional LSTM,nan,0.936829
0,SimpleRNN,0.756457,nan


In [30]:
fig = go.Figure(go.Funnelarea(
    text =results.Model,
    values = results.AUC_Score,
    title = {"position": "top center", "text": "Funnel-Chart of Sentiment Distribution"}
    ))
fig.show()

# BERT

In [31]:
# Loading Dependencies
import os
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
from kaggle_datasets import KaggleDatasets
import transformers

from tokenizers import BertWordPieceTokenizer

In [32]:
# LOADING THE DATA

train1 = pd.read_csv("/kaggle/input/jigsaw-multilingual-toxic-comment-classification/jigsaw-toxic-comment-train.csv")
valid = pd.read_csv('/kaggle/input/jigsaw-multilingual-toxic-comment-classification/validation.csv')
test = pd.read_csv('/kaggle/input/jigsaw-multilingual-toxic-comment-classification/test.csv')
sub = pd.read_csv('/kaggle/input/jigsaw-multilingual-toxic-comment-classification/sample_submission.csv')

In [33]:
def fast_encode(texts, tokenizer, chunk_size=256, maxlen=512):
    """
    Encoder for encoding the text into sequence of integers for BERT Input
    """
    tokenizer.enable_truncation(max_length=maxlen)
    tokenizer.enable_padding(length=maxlen)
    all_ids = []
    
    for i in tqdm(range(0, len(texts), chunk_size)):
        text_chunk = texts[i:i+chunk_size].tolist()
        encs = tokenizer.encode_batch(text_chunk)
        all_ids.extend([enc.ids for enc in encs])
    
    return np.array(all_ids)

In [34]:
#IMP DATA FOR CONFIG

AUTO = tf.data.experimental.AUTOTUNE


# Configuration
EPOCHS = 3
BATCH_SIZE = 16 * strategy.num_replicas_in_sync
MAX_LEN = 192

In [35]:
# First load the real tokenizer
tokenizer = transformers.DistilBertTokenizer.from_pretrained('distilbert-base-multilingual-cased')
# Save the loaded tokenizer locally
tokenizer.save_pretrained('.')
# Reload it with the huggingface tokenizers library
fast_tokenizer = BertWordPieceTokenizer('vocab.txt', lowercase=False)
fast_tokenizer

Tokenizer(vocabulary_size=119547, model=BertWordPiece, unk_token=[UNK], sep_token=[SEP], cls_token=[CLS], pad_token=[PAD], mask_token=[MASK], clean_text=True, handle_chinese_chars=True, strip_accents=None, lowercase=False, wordpieces_prefix=##)

In [36]:
x_train = fast_encode(train1.comment_text.astype(str), fast_tokenizer, maxlen=MAX_LEN)
x_valid = fast_encode(valid.comment_text.astype(str), fast_tokenizer, maxlen=MAX_LEN)
x_test = fast_encode(test.content.astype(str), fast_tokenizer, maxlen=MAX_LEN)

y_train = train1.toxic.values
y_valid = valid.toxic.values

100%|██████████| 250/250 [00:10<00:00, 22.74it/s]


In [37]:
train_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_train, y_train))
    .repeat()
    .shuffle(2048)
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)

valid_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_valid, y_valid))
    .batch(BATCH_SIZE)
    .cache()
    .prefetch(AUTO)
)

test_dataset = (
    tf.data.Dataset
    .from_tensor_slices(x_test)
    .batch(BATCH_SIZE)
)

In [38]:
def build_model(transformer, max_len=512):
    """
    function for training the BERT model
    """
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    sequence_output = transformer(input_word_ids)[0]
    cls_token = sequence_output[:, 0, :]
    out = Dense(1, activation='sigmoid')(cls_token)
    
    model = Model(inputs=input_word_ids, outputs=out)
    model.compile(Adam(lr=1e-5), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [39]:
%%time
with strategy.scope():
    transformer_layer = (
        transformers.TFDistilBertModel
        .from_pretrained('distilbert-base-multilingual-cased')
    )
    model = build_model(transformer_layer, max_len=MAX_LEN)
model.summary()

Some layers from the model checkpoint at distilbert-base-multilingual-cased were not used when initializing TFDistilBertModel: ['vocab_layer_norm', 'vocab_transform', 'activation_13', 'vocab_projector']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-multilingual-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_word_ids (InputLayer)  [(None, 192)]            0         
                                                                 
 tf_distil_bert_model (TFDis  TFBaseModelOutput(last_h  134734080
 tilBertModel)               idden_state=(None, 192,             
                             768),                               
                              hidden_states=None, att            
                             entions=None)                       
                                                                 
 tf.__operators__.getitem (S  (None, 768)              0         
 licingOpLambda)                                                 
                                                                 
 dense_4 (Dense)             (None, 1)                 769       
                                                             

In [40]:
n_steps = x_train.shape[0] // BATCH_SIZE
train_history = model.fit(
    train_dataset,
    steps_per_epoch=n_steps,
    validation_data=valid_dataset,
    epochs=EPOCHS
)

Epoch 1/3
1746/1746 [==============================] - 191s 82ms/step - loss: 0.3356 - accuracy: 0.9031 - val_loss: 0.4354 - val_accuracy: 0.8462
Epoch 2/3
1746/1746 [==============================] - 135s 77ms/step - loss: 0.3168 - accuracy: 0.9043 - val_loss: 0.4444 - val_accuracy: 0.8462
Epoch 3/3
1746/1746 [==============================] - 135s 78ms/step - loss: 0.3164 - accuracy: 0.9043 - val_loss: 0.4470 - val_accuracy: 0.8462


In [41]:
n_steps = x_valid.shape[0] // BATCH_SIZE
train_history_2 = model.fit(
    valid_dataset.repeat(),
    steps_per_epoch=n_steps,
    epochs=EPOCHS*2
)

Epoch 1/6
62/62 [==============================] - 5s 78ms/step - loss: 0.4332 - accuracy: 0.8459
Epoch 2/6
62/62 [==============================] - 32s 76ms/step - loss: 0.4320 - accuracy: 0.8462
Epoch 3/6
62/62 [==============================] - 5s 76ms/step - loss: 0.4313 - accuracy: 0.8460
Epoch 4/6
62/62 [==============================] - 5s 76ms/step - loss: 0.4297 - accuracy: 0.8468
Epoch 5/6
62/62 [==============================] - 5s 76ms/step - loss: 0.4324 - accuracy: 0.8450
Epoch 6/6
62/62 [==============================] - 5s 76ms/step - loss: 0.4318 - accuracy: 0.8454


In [42]:
sub['toxic'] = model.predict(test_dataset, verbose=1)
sub

499/499 [==============================] - 19s 31ms/step


,id,toxic
0,0,0.167015
1,1,0.167017
2,2,0.167021
3,3,0.167017
4,4,0.167017
...,...,...
63807,63807,0.167015
63808,63808,0.167013
63809,63809,0.167000
63810,63810,0.167016


In [43]:
sub.to_csv('submission.csv', index=False)